In [ ]:
import json
import datetime
import requests
import shutil
import os
import time
import pandas as pd

In [ ]:
BASE_PATH = "/mnt/c/Users/chris"
APP_DATA_PATH = "AppData/LocalLow/Wizards Of The Coast/MTGA"
PLAYER_NAME = "spantz"

In [ ]:
deck_data = []
match_data = []
match_commanders = {}

_set_deck_string = "[UnityCrossThreadLogger]==> EventSetDeckV2 "

with open(f"{BASE_PATH}/{APP_DATA_PATH}/Player.log", "r") as f:
    for line in f.readlines():
        if "GameStage_Start" in line:
            line = json.loads(line)

            commander_blob = [
                message for message in line["greToClientEvent"]["greToClientMessages"]
                if message["type"] == "GREMessageType_GameStateMessage"
            ][0]["gameStateMessage"]["gameObjects"]

            match_id = [
                message for message in line["greToClientEvent"]["greToClientMessages"]
                if message["type"] == "GREMessageType_GameStateMessage"
            ][0]["gameStateMessage"]["gameInfo"]["matchID"]
            
            if match_id not in match_commanders.keys():
                match_commanders[match_id] = [{
                    "player_seat_id": player["ownerSeatId"],
                    "commander": player["grpId"]
                } for player in commander_blob]
        elif line.startswith(_set_deck_string):
            # 🥲
            # but can also save off the full decklist if we want from this
            deck_data.append(
                json.loads(
                    json.loads(
                        line[(len(_set_deck_string)):].replace("\n", "")
                    )["request"]
                )
            )
        elif '"stateType": "MatchGameRoomStateType_MatchCompleted"' in line:
            match_data.append(json.loads(line))

In [ ]:
assert len(deck_data) == len(match_data)

In [ ]:
print(len(match_data), len(deck_data))

In [ ]:
match_data_parsed = []

for deck, match in zip(deck_data, match_data):
    player_seat = [
        player["teamId"] for player in match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["gameRoomConfig"]["reservedPlayers"]
        if player["playerName"] == PLAYER_NAME
    ][0]
    
    _match_data = {
        "game_id": match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["gameRoomConfig"]["matchId"],
        "timestamp": datetime.datetime.fromtimestamp(int(match["timestamp"]) / 1000),
        "deck_name": deck["Summary"]["Name"],
        "opponent": [
            player["playerName"] for player in match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["gameRoomConfig"]["reservedPlayers"]
            if player["playerName"] != PLAYER_NAME
        ][0],
        "game_won": [
            result.get("winningTeamId", None) for result in match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["finalMatchResult"]["resultList"]
            if result["scope"] == "MatchScope_Match"
        ][0] == player_seat,
        "game_result_reason": [
            result["reason"] for result in match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["finalMatchResult"]["resultList"]
            if result["scope"] == "MatchScope_Match"
        ][0],
        "player_seat": player_seat,
        "opponent_seat": 3 - player_seat,
    }

    match_data_parsed.append(_match_data)

In [ ]:
match_df = pd.DataFrame(match_data_parsed)

In [ ]:
match_df

In [ ]:
def get_scryfall_name_from_arena_id(id: int):
    resp = json.loads(requests.get(f"https://api.scryfall.com/cards/arena/{id}").text)
    return resp.get("name", id)

def get_player_commanders(x):
    _match = x["game_id"]
    try:
        result = pd.Series([
            get_scryfall_name_from_arena_id([
                player["commander"] for player in match_commanders[_match]
                if player["player_seat_id"] == x["player_seat"]
            ][0]),
            get_scryfall_name_from_arena_id([
                player["commander"] for player in match_commanders[_match]
                if player["player_seat_id"] == x["opponent_seat"]
            ][0])
        ])
    except KeyError:
        # In cases of a player disconnect or backend issue when matching,
        # the match can immediately result in a draw with no player commander
        # data ever being returned to the client
        result = pd.Series([None, None])

    return result
    
match_df[["player_commander", "opponent_commander"]] = match_df.apply(lambda x: get_player_commanders(x), axis=1)

In [ ]:
match_df = match_df.drop(["player_seat", "opponent_seat"], axis=1)

In [ ]:
match_df

In [ ]:
sum(match_df["game_won"])/match_df.shape[0]

In [ ]:
if os.path.exists("match_history/mtga_match_log.csv"):
    match_df.to_csv(f"match_history/mtga_match_log.csv", index=False, header=False, mode="a")
else:
    os.makedirs("match_history", exist_ok=True)
    match_df.to_csv(f"match_history/mtga_match_log.csv", index=False, header=True, mode="w")

In [ ]:
source_path = f"{BASE_PATH}/{APP_DATA_PATH}/Player.log"
destination_path = f"{BASE_PATH}/{APP_DATA_PATH}/Player_{time.time_ns()}.log"

# Ensure the source file exists
if os.path.exists(source_path):
    shutil.move(source_path, destination_path)
    print(f"File moved from {source_path} to {destination_path}")
else:
    print(f"Source file {source_path} not found.")